In [21]:
import cv2
import numpy as np

In [22]:
boundary_x = 700
threshold = 10_000

In [23]:
cap = cv2.VideoCapture('../resources/input/in_out_2.mov')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
back_sub_alg = cv2.createBackgroundSubtractorMOG2()
frame_height

720

In [24]:
frame_size  = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 2*int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
output = cv2.VideoWriter('../resources/output/in_out_2.avi', fourcc, frame_rate, frame_size)

In [25]:
i = 0
while i < 281:
    # print(i)
    hand_in = False
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    back_sub_mask = back_sub_alg.apply(frame_gray)
    kernel = np.ones((6, 6),np.uint8)
    back_sub_mask = cv2.morphologyEx(back_sub_mask, cv2.MORPH_OPEN, kernel)
    _, binary_mask = cv2.threshold(back_sub_mask, 250, 255, cv2.THRESH_BINARY)
    if np.sum(binary_mask[:, boundary_x:]==255) > threshold:
        hand_in = True
    annotated = cv2.line(frame, (boundary_x, 0), (boundary_x, frame_height), (255, 255, 0), 3)
    text = "Hand In" if hand_in else "Hand Out"
    text_overlay = cv2.putText(frame, text, (50, 100), 0, 3, (255, 255, 255))
    merged_binary = cv2.merge((binary_mask, binary_mask, binary_mask))
    combined_frame = cv2.vconcat([frame, merged_binary])
    output.write(combined_frame)
    # cv2.imshow("Frame", text_overlay)
    # cv2.waitKey(0)
    i += 1
output.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

    